In [1]:
import pandas as pd
import numpy as np
import re
import os

datasets_dir = os.getcwd() + '/data/'

In [2]:
contributions_2016 = 'contributions-2015-2016.csv'
df_contributions_2016 = pd.read_csv(datasets_dir + contributions_2016, encoding = 'latin1')

In [3]:
statereps_2016 = 'State_Reps_2015-2016.csv'
df_reps_2016 = pd.read_csv(datasets_dir + statereps_2016, encoding = 'latin1')

In [4]:
contributions_2018 = 'contributions-2017-2018.csv'
df_contributions_2018 = pd.read_csv(datasets_dir + contributions_2018, encoding = 'latin1')

C:\Users\WINPC\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
statereps_2018 = 'State_Reps_2017-2018.csv'
df_reps_2018 = pd.read_csv(datasets_dir + statereps_2018, encoding = 'latin1')

In [6]:
real_estate_companies = 'real_estate_companies.txt'
df_real_estate = pd.read_csv(datasets_dir + real_estate_companies, encoding = 'latin1')

In [7]:
df_contributions_2016 = df_contributions_2016.merge(df_reps_2016, how='left', left_on='Full_Name', right_on='Representative')
df_contributions_2018 = df_contributions_2018.merge(df_reps_2018, how='left', left_on='Full_Name', right_on='Representative')

In [8]:
import pandas as pd

# Get list of all elected Candidates from 2015-2018
reps = [df_reps_2016, df_reps_2018]
df_reps = pd.concat(reps, sort=True)
state_reps = list(df_reps['Representative'].unique())

# Deallocate memory
reps = []
df_reps_2016 = []
df_reps_2018 = []

In [9]:
import pandas as pd

# Combine all contributions from 2015-2018
contributions = [df_contributions_2016, df_contributions_2018]
df_contributions = pd.concat(contributions, sort=True)

# Deallocate memory
contributions = []
df_contributions_2016 = []
df_contributions_2018 = []

In [10]:
display(len(state_reps))
display(df_contributions.shape)

193

(894666, 22)

In [11]:
df_contribution_state_reps = df_contributions.loc[df_contributions.Full_Name.isin(state_reps)]

In [12]:
# Creating new Donor_Name column that will allow easier analysis here
# and moving forward

import math

def donor_name(first_name, last_name):
    donor_name = ''
    if (type(first_name) == str):
        donor_name += first_name.upper().replace(":-,.'&", ' ') + ' '
    if (type(last_name) == str):
        donor_name += last_name.upper().replace(":-,.'&", ' ')
    return donor_name

df_contribution_state_reps['Donor_Name'] = df_contribution_state_reps[['First_Name', 'Last_Name']].apply(lambda x: donor_name(x[0], x[1]), axis=1)

C:\Users\WINPC\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
import re
import numpy as np

# Regular Expression Search for each Industry

# Law Enforcement
law_enforcement_keywords = ['law enforcement', 'district attorney', 'prosecutor', 'detective', 'trooper', 'probation officer', 'sheriff', 'correction', 'prison', 'patrolm[ae]n', 'parole', 'court', 'trial', 'firefighter', 'fire fighter']
law_enforcement_regex = ''

for line in law_enforcement_keywords:
    law_enforcement_regex += '(' + line +')|'

law_enforcement_regex += '(police)'
# display(regex_expr)

In [14]:
# Education
education_keywords = ['teacher', 'professor', 'lecture', 'educator', 'university', 'school', 'college', 'tutor', 'tuition', 'academ(y|ic)']
education_regex = ''

for line in education_keywords:
    education_regex += '(' + line +')|'

education_regex += '(education)'
# display(regex_expr)

In [15]:
# Real Estate
realestate_keywords = ['realtor', 'realty', 'housing', 'build', 'leasing', '[^(it)|^(software )|^(data )]developer', '[^(it )|^(software )|^(data )]architect', 'construction', 'lodging', 'propert(y|ies)', 'residential', 'contractor', 'pile drive']
realestate_regex = ''

for line in realestate_keywords:
    realestate_regex += '(' + line +')|'

# for row in df_real_estate:
#     row.lower().replace(":-,.'&", ' ')

realestate_regex += '(real estate)'
# display(regex_expr)

In [16]:
# Healthcare
# Note - 'ophthalm', 'optometr', 'urolog', 'chiropract', 'psychiatr', 'psycholog' are added to match root words of medical specializations
healthcare_keywords = ['health care', 'health', 'medical', 'hospital', 'nurse', 'rehab', 'nursing', 'clinic', 'doctor', 'psycholog', 'surgeon', 'dental', 'dentist', 'phys[i]+cian', 'op[h]+thalm', 'urolog', 'pharmac', 'optometr', 'eye care', 'eye center', 'chiropract', 'psychiatr', 'podiatr', '\wem[ts]\w', 'ambul[ae]nce', 'emergency']
# , '\wrn\w'
healthcare_regex = ''

for line in healthcare_keywords:
    healthcare_regex += '(' + line +')|'

healthcare_regex += '(healthcare)'
# display(regex_expr)

In [17]:
# BioTech / Pharma
biotech_keywords = ['genetech', 'bioscience', 'pharma']
biotech_regex = ''

for line in biotech_keywords:
    biotech_regex += '(' + line +')|'

biotech_regex += '(bio science)'
# display(regex_expr)

In [18]:
# Utilities
energy_keywords = ['eversource', 'sunpower', 'grid', 'wind', 'solar']
energy_regex = ''

for line in energy_keywords:
    energy_regex += '(' + line +')|'

energy_regex += '(energy)'
# display(regex_expr)

In [19]:
# Transport
transport_keywords = ['(?!pile )drive', 'teamster', 'truck', 'motor', 'ferry', 'cruise', 'air[ ]*port', 'sea[ ]*port']
transport_regex = ''

for line in transport_keywords:
    transport_regex += '(' + line +')|'

transport_regex += '(transport)'
# display(regex_expr)

In [20]:
# Retirees
retiree_keywords = ['retire']
retiree_regex = ''

for line in retiree_keywords:
    retiree_regex += '(' + line +')|'

retiree_regex += '(retire)'
# display(regex_expr)

In [21]:
# Self-employed
self_employed_keywords = ['^self$', 'owner']
self_employed_regex = ''

for line in self_employed_keywords:
    self_employed_regex += '(' + line +')|'

self_employed_regex += '(self employ)'
# display(regex_expr)

In [22]:
# Unemployed
unemployed_keywords = ['not employ', 'at home', 'none', '^0$', 'home[ ]*maker', 'house[ ]*wife']
unemployed_regex = ''

for line in unemployed_keywords:
    unemployed_regex += '(' + line +')|'

unemployed_regex += '(unemploy)'
# display(regex_expr)

In [23]:
# Government
government_keywords = ['commonwealth of ma', 'reimbursement']
government_regex = ''

for line in government_keywords:
    government_regex += '(' + line +')|'

government_regex += '(government)'
# display(regex_expr)

In [24]:
import numpy as np

def find_industry(donor_name, occupation, employer):
    text = str(donor_name) + ' ' + str(occupation) + ' ' + str(employer)
    text = text.lower().replace(":-,.'&", ' ')
#     display(text)
    
    if(re.search(law_enforcement_regex, text)):
        return 'Law Enforcement'
    if (re.search(education_regex, text)):
        return 'Education'
    if (re.search(realestate_regex, text)):
        return 'Real Estate'
    if (re.search(healthcare_regex, text)):
        return 'Healthcare'
    if (re.search(biotech_regex, text)):
        return 'Biotech/Pharma'
    if (re.search(energy_regex, text)):
        return 'Utilities'
    if (re.search(retiree_regex, text)):
        return 'Retired'
    if (re.search(self_employed_regex, text)):
        return 'Self-Employed'
    if (re.search(unemployed_regex, text)):
        return 'Unemployed'
    if(re.search(government_regex, text)):
        return 'Government'
    if(re.search(transport_regex, text)):
        return 'Transportation'
    
    for row in df_real_estate['Company']:
        if (type(employer)==str):
            found = fuzzy_compare(employer, row)
            if (found):
                return 'Real Estate'
        if (type(donor_name)==str):
            found = fuzzy_compare(donor_name, row)
            if (found):
                return 'Real Estate'
    
    if(type(occupation)==float and type(employer)==float):
        if(np.isnan(occupation) and np.isnan(employer)):
            return 'Unreported'
    
    # Case no match found
    return np.nan

In [25]:
# PAC keywords
PAC_keywords = ['\Wpac\W', 'pol action', 'political action', 'committee', 'action']
PAC_regex = ''

for line in PAC_keywords:
    PAC_regex += '(' + line +')|'

PAC_regex += '(real estate)'
# display(regex_expr)

In [26]:
# Union Keywords
union_keywords = ['\Wunion\W', 'association']
union_regex = ''

for line in union_keywords:
    union_regex += '(' + line +')|'

union_regex += '(union pac)'
# display(regex_expr)

In [27]:
# Lobbyist Keywords
lobbyist_keywords = ['lobbyist']
lobbyist_regex = ''

for line in lobbyist_keywords:
    lobbyist_regex += '(' + line +')|'

lobbyist_regex += '(lobby)'
# display(regex_expr)

In [28]:
# Candidate
candidate_keywords = []
candidate_regex = ''

for line in candidate_keywords:
    candidate_regex += '(' + line +')|'

candidate_regex += '(reimbursement)'
# display(regex_expr)

In [29]:
from fuzzywuzzy import fuzz

def fuzzy_compare(str1, str2):
    str1 = str(str1)
    str2 = str(str2)
    ratio = fuzz.ratio(str1.lower().replace(":-,.'&", ' '), str2.lower().replace(":-,.'&", ' '))
    if (ratio > 85):
        return True
    else:
        return False

In [30]:
import pandas as pd

df_state_reps = pd.DataFrame(state_reps, columns = ['Representative'])
# display(df_state_reps['Representative'].head())

def find_PAC(donor_name, candidate_name): 
    text = ' ' + str(donor_name) + ' '
    text = text.lower().replace(":-,.'&", ' ')
#     display(text)
    # Note - Union checking must come before PAC because of "union pac"
    if(re.search(union_regex, text)):
        return 'Union'
    if(re.search(PAC_regex, text)):
        return 'PAC'
    if(re.search(candidate_regex, text)):
        return 'Candidate'
    
    candidateFound = fuzzy_compare(donor_name, candidate_name)
    if (candidateFound):
        return 'Candidate'
    
    # Case no match found
    return np.nan

In [31]:
df_contribution_state_reps['Industry'] = df_contribution_state_reps[['Donor_Name', 'Occupation', 'Employer']].apply(lambda x: find_industry(x[0], x[1], x[2]), axis=1)

C:\Users\WINPC\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
df_contribution_state_reps['PAC'] = df_contribution_state_reps[['Donor_Name', 'Full_Name']].apply(lambda x: find_PAC(x[0], x[1]), axis=1)

C:\Users\WINPC\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
import re

def clean_text(text):
    '''
    Remove special charecters, numbers
    Return lower case alphabet charecters only
    '''
    text = str(text).upper()
    text = re.sub('[^A-Za-z]+', '', text)
    return text

def takeoff_par(name):
    name = str(name)
    head, sep, tail = name.partition('(')
    return head

In [34]:
# Generate list of Lobbyist names

import pandas as pd

df_lobby_2016 = pd.read_csv('data/lobbyist(2015-2016).csv')
df_lobby_2018 = pd.read_csv('data/lobbyist(2017-2018).csv')

lobby_list = df_lobby_2016.append(df_lobby_2018)['Name'].unique()
donor_list = df_contribution_state_reps['Donor_Name'].unique()

lobby_list = [takeoff_par(x) for x in lobby_list]
lobby_list_clean = [clean_text(x) for x in lobby_list]
donor_list_clean = [clean_text(x) for x in donor_list]

match = []
donor_list_clean = np.array(donor_list_clean)
lobby_list_clean = np.array(lobby_list_clean)
for donor in donor_list_clean:
    for lobby in lobby_list_clean:
        if fuzz.ratio(donor, lobby) >= 85:
            match.append(donor)
            
match_un = np.array(match.copy())
match_set = set(match.copy())

df_match_set = pd.DataFrame(match_set, columns=['Lobbyist'])
df_match_set.to_csv(os.getcwd() +  '/data/lobbyist_names.csv', header=True, index=False)

In [35]:
df_contribution_state_reps['Donor_clean'] = df_contribution_state_reps['Donor_Name'].apply(clean_text)

C:\Users\WINPC\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
import pandas as pd

df_match_set = pd.read_csv(os.getcwd() +  '/data/lobbyist_names.csv')

def lobby_search(name):
    return name in df_match_set['Lobbyist'].values

In [37]:
df_contribution_state_reps['Lobbyist'] = df_contribution_state_reps['Donor_clean'].apply(lobby_search)
# df_contribution_state_reps.drop(columns=['Donor_clean'], inplace=True)

C:\Users\WINPC\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [38]:
df_contribution_state_reps.to_csv(os.getcwd() +  '/data/contribution_processed.csv')